## imports

In [247]:
import pickle
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import cv2
import numpy as np

# Part 1

## load dataset

In [248]:
def load_dataset():
  image_pickle_file_path = 'images.pkl'
  label_pickle_file_path = 'label.pkl'

  with open(image_pickle_file_path, 'rb') as file:
    images = pickle.load(file)

  with open(label_pickle_file_path, 'rb') as file:
    labels = pickle.load(file)

  return images, labels

In [249]:
n_images = 20

In [250]:
images, labels = load_dataset()
labels = np.array(labels)
random_indices = np.random.choice(560, size=n_images, replace=False)
# Extract the randomly selected values
random_values = images[random_indices]
random_labels = labels[random_indices]
from dataclasses import dataclass, field


@dataclass
class Images:
    image: np.ndarray = None
    original_features = []
    features = []
    clusters = None

## Proccess on images
- Extract features
- normalize features
- make features a single dimnetion vector

In [251]:
def extract_features(image):
    features = []
    original_features = []

    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # For each pixel in the image
    for i in range(img_hsv.shape[0]):
        for j in range(img_hsv.shape[1]):
            pixel = img_hsv[i, j]
            h, s, v = pixel
            x, y = i, j
            
            features.append((h, s, v, x, y))

    return features

In [252]:
def normalize_features(features):
    features_array = np.array(features)
    scaler = MinMaxScaler()
    normalized_features_array = scaler.fit_transform(features_array)
    normalized_features = normalized_features_array.tolist()
    return normalized_features

In [253]:
new_features = []

for img in random_values:
    features = extract_features(img)
    new_features.append(features)

In [254]:
# normalize features
for i, fimg in enumerate(new_features):
    new_features[i] = normalize_features(fimg)

In [255]:
def set_priority(arr, color, pos):
    color_columns = np.tile(arr[:, :3], (1, color))
    pos_columns = np.tile(arr[:, 3:], (1, pos))
    duplicated_arr = np.concatenate((color_columns, pos_columns), axis=1)

    return duplicated_arr

In [256]:
new_features_freq = []
for nf in new_features:
    new_features_freq.append(set_priority(np.array(nf), 4, 2).tolist())

## clustering

### k-means

In [257]:
k_number = 15
kmeans = KMeans(n_clusters=k_number)
new_clusters = []
for nf in new_features_freq:
    clusters = kmeans.fit_predict(nf)
    new_clusters.append(clusters)

c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\clus

In [258]:
for i, nc in enumerate(new_clusters):
    new_clusters[i] = nc.reshape(random_values.shape[1], random_values.shape[2])

## extracting clusters features for each image sepratedly

In [259]:
def find_matching_triads(array1, array2):   

    unique_values = np.unique(array1)
    result = []

    for value in unique_values:
        indexes = np.where(array1 == value)[0]
        triads = array2[indexes, :].tolist()
        result.append(triads)
        
    return result

### mean color for each cluster in each image

In [260]:
def calculate_mean(data):

    data_array = np.array(data)
    mean_values = np.mean(data_array, axis=0)
    mean_values = mean_values.astype(int)
    mean_values_list = mean_values.tolist()
    
    return mean_values_list

In [261]:
def mean_clusters_each_image(new_cls, random_values):

    all_images_mean_values = []
    
    for i, image_cls in enumerate(new_cls):

        one_image_clusters_mean_values = []
        reshaped_image_cls = image_cls.reshape(image_cls.shape[0] * image_cls.shape[1])
        reshaped_random_value = random_values[i].reshape(random_values.shape[1] * random_values.shape[2], random_values.shape[3])

        similar_clusters_pixels = find_matching_triads(reshaped_image_cls, reshaped_random_value)

        for smp in similar_clusters_pixels:
            one_image_clusters_mean_values.append(calculate_mean(smp))

        all_images_mean_values.append(one_image_clusters_mean_values)

    return all_images_mean_values

In [262]:
mean_each_cluster_each_image = mean_clusters_each_image(new_clusters, random_values)

## display clustering

In [263]:
def display_clustering(image_clusters, each_cluster_color, image_index):
    # Create an empty array to hold the HSV values for each pixel
    rgb_image = np.zeros((image_clusters.shape[0], image_clusters.shape[1], 3), dtype=np.uint8)

    # Assign the corresponding HSV values to each pixel
    for i in range(image_clusters.shape[0]):
        for j in range(image_clusters.shape[1]):
            value = image_clusters[i, j] 
            hsv = each_cluster_color[value]
            # rgb = colorsys.hsv_to_rgb(hsv[0] / 360, hsv[1] / 100, hsv[2] / 100)
            # rgb_image[i, j] = np.round(np.array(rgb) * 255).astype(int)
            rgb_image[i, j] = hsv

    # Display the image with corresponding RGB values using cv2.imshow
    cv2.imshow('org', random_values[image_index])
    cv2.imshow('Image with Corresponding RGB Values', cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR))

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [264]:
each_cluster_color = [
    [255, 0, 0],    # Red
    [0, 255, 0],    # Green
    [0, 0, 255],    # Blue
    [255, 255, 0],  # Yellow
    [255, 0, 255],  # Magenta
    [0, 255, 255],  # Cyan
    [128, 0, 0],    # Maroon
    [0, 128, 0],    # Green (Dark)
    [0, 0, 128],    # Navy
    [128, 128, 128], # Gray
    [255, 128, 0],  # Orange
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [128, 128, 0],  # Olive
    [192, 192, 192]  # Silver
]

In [265]:
# for i, img_clusters in enumerate(new_clusters[:2]):
#     display_clustering(img_clusters, each_cluster_color, i)

# Part 2

## create clusters feature vectors 

In [266]:
def extract_cluster_features(image_number, image_clusters, arr_new_features, k_number):
    clusters_features = []
    clusters = [[] for i in range(k_number)]
    for index, value in enumerate(image_clusters):
        clusters[value].append(arr_new_features[image_number, index])
    for cluster in clusters: # clusters contains all pixels which belong to the same cluster.
        temp_array = np.array(cluster)
        min_value = np.min(temp_array, axis=0)
        max_value = np.max(temp_array, axis=0)
        mean_value = np.mean(temp_array, axis=0)
        x_field = max_value[3] - min_value[3]
        y_field = max_value[4] - min_value[4]
        shape_field = np.array([x_field / y_field])
        combined_results = np.concatenate((min_value[:3], max_value[:3], mean_value[:3], shape_field), axis=None)
        clusters_features.append(combined_results)

    return np.array(clusters_features)

In [267]:
all_images_clusters_features = []
arr_new_features = np.array(new_features)
for image_num, image_clusters in enumerate(new_clusters):
    reshaped_image_clusters = image_clusters.reshape(image_clusters.shape[0] * image_clusters.shape[1])
    all_images_clusters_features.append(extract_cluster_features(image_num, 
                                                                 reshaped_image_clusters,
                                                                 arr_new_features, k_number))

In [268]:
all_images_clusters_features

[array([[0.39325843, 0.00784314, 0.01176471, 1.        , 0.42352941,
         0.53333333, 0.71818339, 0.1859322 , 0.24711091, 1.14673913],
        [0.        , 0.49803922, 0.00392157, 0.71910112, 1.        ,
         0.52941176, 0.21355188, 0.76332945, 0.24489891, 0.66331658],
        [0.42134831, 0.00784314, 0.20784314, 1.        , 0.79607843,
         0.74509804, 0.71424788, 0.35290583, 0.49092497, 1.26736111],
        [0.        , 0.        , 0.        , 0.25280899, 0.28627451,
         0.65490196, 0.02145189, 0.01377508, 0.05327411, 0.90952381],
        [0.        , 0.        , 0.        , 0.50561798, 0.77254902,
         0.78039216, 0.2254924 , 0.34316058, 0.24074084, 0.77399381],
        [0.        , 0.5254902 , 0.00392157, 0.84269663, 1.        ,
         0.48627451, 0.26084827, 0.80784633, 0.07330718, 0.62449799],
        [0.28089888, 0.01176471, 0.00784314, 0.98314607, 0.8       ,
         0.4       , 0.60792083, 0.26143438, 0.11567637, 1.48787879],
        [0.        , 0.    

## set priority for clusters features

In [269]:
color_pri = 5
pos_pri = 6

In [270]:
def set_priority_clusters_vectors(arr, color, pos):
    color_columns = np.tile(arr[:, :9], (1, color))
    pos_columns = np.tile(arr[:, 9:], (1, pos))
    duplicated_arr = np.concatenate((color_columns, pos_columns), axis=1)

    return duplicated_arr

In [271]:
# all_images_clusters_features_freq = []
# for nf in all_images_clusters_features:
#     all_images_clusters_features_freq.\
#             append(set_priority_clusters_vectors(np.array(nf), color_pri, pos_pri).tolist())

In [272]:
# all_images_clusters_features_freq

## clustering clusters

### k-means

In [273]:
# k_number_2 = 100
# kmeans_2 = KMeans(n_clusters=k_number_2)
# arr_all_images_clusters_features_freq = np.array(all_images_clusters_features_freq)
# arr_all_images_clusters_features_freq = arr_all_images_clusters_features_freq.reshape(\
#                                             arr_all_images_clusters_features_freq.shape[0] *\
#                                             arr_all_images_clusters_features_freq.shape[1],
#                                             arr_all_images_clusters_features_freq.shape[2])

# clusters2 = kmeans_2.fit_predict(arr_all_images_clusters_features_freq)

In [274]:
# clusters2_arr = clusters2.reshape(np.array(all_images_clusters_features_freq).shape[0], np.array(all_images_clusters_features_freq).shape[1])
# clusters2_arr

In [275]:
def count_numbers(lst, k_number_2):
    # Initialize a list to store counts
    count_list = [0] * k_number_2  # Index 0 is not used to align with numbers 1 to 10

    # Count occurrences of each number in the list
    for num in lst:
        if 1 <= num <= 10:
            count_list[num - 1] += 1  # Adjust index to align with numbers 1 to 10

    return count_list

In [276]:
# histogram = np.empty((n_images, k_number_2))
# for i, img_cls in enumerate(clusters2_arr):
#     histogram[i] = count_numbers(img_cls, k_number_2)

# histogram

## classify

In [277]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Classify the datapoints with the Random Forest Classifier
def classify(datapoints, labels):
  test_size = 0.2
  X_train, X_test, y_train, y_test = train_test_split(datapoints, labels, test_size=test_size, random_state=42)

  clf = RandomForestClassifier(n_estimators=100, random_state=42)

  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)

  y_prob = clf.predict_proba(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  # print(f"Accuracy: {accuracy * 100:.2f}%")

  # Print the true labels and predicted labels
  # print("True labels:")
  # print(y_test)

  # print("Predicted labels:")
  # print(y_pred)

  # print("Probability estimates:")
  # print(y_prob)

  return [accuracy, y_test, y_pred, y_prob]

In [278]:
# classify(histogram, random_labels)

# Phase 2

## create images feature vectors

### create without removing clusters

In [279]:
"""
    Extract a features for each image 
    by calculating the mean of all its culsters
"""
def calculate_mean_imgs_clstrs_features():
    histograms = []
    for ftr_clstrs_img in all_images_clusters_features:
        histograms.append(np.mean(ftr_clstrs_img, axis=0))
    return histograms

In [280]:
# histograms = calculate_mean_imgs_clstrs_features()

### create with removing clusters

In [287]:
"""
    Extract a features for each image 
    by calculating the mean of its culsters excpet one of them
"""
def calculate_mean_imgs_clstrs_excpet_index(index_, img_num):
    histograms = []
    for i, ftr_clstrs_img in enumerate(all_images_clusters_features):
        if i == img_num:
            histograms.append(np.mean(np.concatenate((ftr_clstrs_img[:index_], ftr_clstrs_img[index_+1:])), axis=0))
        else:
            histograms.append(np.mean(ftr_clstrs_img, axis=0))  

    return histograms

In [288]:
def cal_classify_results(img_num):
    classify_results = []
    for clstr_num in range(k_number):
        histograms = calculate_mean_imgs_clstrs_excpet_index(index_= clstr_num, img_num = 1)
        rslt = classify(histograms, random_labels)
        classify_results.append(rslt)
    return classify_results

In [289]:
classify_results = cal_classify_results(0)

In [290]:
classify_results

[[0.0,
  array([0, 5, 3, 5]),
  array([4, 1, 0, 1]),
  array([[0.14, 0.21, 0.13, 0.12, 0.23, 0.  , 0.17],
         [0.28, 0.37, 0.09, 0.05, 0.1 , 0.06, 0.05],
         [0.61, 0.08, 0.03, 0.15, 0.08, 0.01, 0.04],
         [0.18, 0.3 , 0.01, 0.07, 0.17, 0.19, 0.08]])],
 [0.0,
  array([0, 5, 3, 5]),
  array([4, 1, 0, 1]),
  array([[0.14, 0.21, 0.13, 0.12, 0.23, 0.  , 0.17],
         [0.28, 0.37, 0.09, 0.05, 0.1 , 0.06, 0.05],
         [0.61, 0.08, 0.03, 0.15, 0.08, 0.01, 0.04],
         [0.26, 0.31, 0.01, 0.07, 0.08, 0.24, 0.03]])],
 [0.0,
  array([0, 5, 3, 5]),
  array([4, 1, 0, 1]),
  array([[0.14, 0.21, 0.13, 0.12, 0.23, 0.  , 0.17],
         [0.28, 0.37, 0.09, 0.05, 0.1 , 0.06, 0.05],
         [0.61, 0.08, 0.03, 0.15, 0.08, 0.01, 0.04],
         [0.24, 0.34, 0.01, 0.08, 0.11, 0.18, 0.04]])],
 [0.0,
  array([0, 5, 3, 5]),
  array([4, 1, 0, 1]),
  array([[0.14, 0.21, 0.13, 0.12, 0.23, 0.  , 0.17],
         [0.28, 0.37, 0.09, 0.05, 0.1 , 0.06, 0.05],
         [0.61, 0.08, 0.03, 0.15, 0.0

### gather the labels probability of each image

In [294]:
def cal_probs():
    probs = [ [] for _ in range(len(classify_results[0][1])) ]
    for result in classify_results:
        for i, imgs_probs in enumerate(result[3]):
            probs[i].append(imgs_probs)
    
    return(probs)

In [295]:
probs = cal_probs()

### check the lables probability of each image

In [296]:
for i, prob in enumerate(probs):
    print(np.array(prob))
    print(f"acc={classify_results[i][0]}, true labels={classify_results[i][1]}, predicted labels={classify_results[i][2]}")
    print("---------------------------------")

[[0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]
 [0.14 0.21 0.13 0.12 0.23 0.   0.17]]
acc=0.0, true labels=[0 5 3 5], predicted labels=[4 1 0 1]
---------------------------------
[[0.28 0.37 0.09 0.05 0.1  0.06 0.05]
 [0.28 0.37 0.09 0.05 0.1  0.06 0.05]
 [0.28 0.37 0.09 0.05 0.1  0.06 0.05]
 [0.28 0.37 0.09 0.05 0.1  0.06 0.05]
 [0.28 0.37 0.09 0.05 0.1  0.06 0.05]
 [0.28 0.37 0.09 0.05 0.1  0.06 0.05]
 [0.28 0.37 0.09 0.05 0.1  0.06 0.05]
 [0.28 0.37 0.09 0.05 0.1  0.06 0.05]
 [0.28 0.37 0.09 0.05 0.1  0.06 

In [285]:
# classify(histograms, random_labels)